In [1]:
import sys
import os
import json

from hana_ml.dataframe import ConnectionContext

In [3]:
with open(os.path.join(os.getcwd(), './env_cloud.json')) as f:
    hana_env_c = json.load(f)
    port_c  = hana_env_c['port']
    user_c  = hana_env_c['user']
    url_c  = hana_env_c['url']
    pwd_c  = hana_env_c['pwd']

cc = ConnectionContext(url_c, port_c, user_c, pwd_c)
print(cc.hana_version())
print(cc.get_current_schema())

4.00.000.00.1624913057 (fa/CE2021.18)
GRAPH_USER


## Create Graph from Vertices and Edges

In [4]:
from hana_ml.dataframe import create_dataframe_from_pandas
import pandas as pd

# example SHAPE column: POINT (1752440.6821975708 5439964.327102661)
v_hdf = create_dataframe_from_pandas(
    connection_context=cc,
    pandas_df=pd.read_csv("./datasets/wwc_stormwater_vertices.csv"),
    table_name="LM_STROM_WATER_VERTICES",
    primary_key="ID",
    geo_cols=["SHAPE"],
    srid=2193,
    force=True)

# example SHAPE column: LINESTRING (1749169.286201477 5422260.568099976, 1749162.987197876 5422242.643096924)
e_hdf = create_dataframe_from_pandas(
    connection_context=cc,
    pandas_df=pd.read_csv("./datasets/wwc_stormwater_edges.csv"),
    table_name="LM_STORM_WATER_EDGES",
    primary_key="ID",
    not_nulls=["SOURCE", "TARGET"],
    geo_cols=["SHAPE"],
    srid=2193,
    force=True)

100%|██████████| 1/1 [00:07<00:00,  7.81s/it]


In [5]:
import hana_ml.graph as hg

g_storm = hg.create_graph_from_dataframes(
    connection_context=cc,
    vertices_df=v_hdf,
    vertex_key_column="ID",
    edges_df=e_hdf,
    edge_source_column="SOURCE",
    edge_target_column="TARGET",
    edge_key_column="ID",
    workspace_name="LM_STORM_WATER",
    )

In [8]:
# Let's check
hgw = hg.discover_graph_workspaces(cc)
hgw[hgw.WORKSPACE_NAME == 'LM_STORM_WATER']

,SCHEMA_NAME,WORKSPACE_NAME,IS_VALID,EDGE_SCHEMA_NAME,EDGE_TABLE_NAME,EDGE_KEY_COLUMN_NAME,EDGE_TARGET_COLUMN,EDGE_SOURCE_COLUMN_NAME,VERTEX_SCHEMA_NAME,VERTEX_TABLE_NAME,VERTEX_KEY_COLUMN_NAME
12,GRAPH_USER,LM_STORM_WATER,NaN,GRAPH_USER,LM_STORM_WATER_EDGES,ID,TARGET,SOURCE,GRAPH_USER,LM_STROM_WATER_VERTICES,ID


In [9]:
# Instantiate existing graph
# here only for demo purposes, since we already instantiated it during creation
g_storm = hg.Graph(
    connection_context=cc,
    workspace_name='LM_STORM_WATER',
    schema="GRAPH_USER"  # Optional, only needed if the schema differs from your logon schema
)

In [10]:
print(g_storm)


Workspace schema: GRAPH_USER 
Workspace name: LM_STORM_WATER

Vertex table schema: GRAPH_USER
Vertex table name: LM_STROM_WATER_VERTICES
Vertex table key column: ID
Vertex table key column dtype: NVARCHAR(5000)
Vertex table SQL statement: SELECT * FROM "GRAPH_USER"."LM_STROM_WATER_VERTICES"

Edge table schema: GRAPH_USER
Edge table name: LM_STORM_WATER_EDGES
Edge table key column: ID
Edge table key column dtype: INT
Edge table source columns: SOURCE
Edge table target column: TARGET
Edge table SQL statement: SELECT * FROM "GRAPH_USER"."LM_STORM_WATER_EDGES"



## Let's plot the whole graph on a map

In [11]:
from keplergl import KeplerGl

pdf_storm_edges = g_storm.edges_hdf.select('ID', ('SHAPE_GEO.ST_TRANSFORM(4326).ST_ASGEOJSON()', 'GJ')).collect()

map = KeplerGl(height=600, width=800)
map.add_data(pdf_storm_edges, 'Stormwater')
map

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'Stormwater': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, …

## Inspect some basic graph stats

In [12]:
g_storm.describe()

COUNT(VERTICES)         50638
COUNT(EDGES)            43003
COUNT(SELF_LOOPS)           3
MIN(OUT_DEG)                0
MIN(IN_DEG)                 0
MIN(DEG)                    1
MAX(OUT_DEG)                6
MAX(IN_DEG)                 8
MAX(DEG)                    9
AVG(OUT_DEG)         0.955155
AVG(IN_DEG)          0.955155
AVG(DEG)              1.91031
DENSITY              0.000016
COUNT(TRIANGLES)           98
IS_CONNECTED            False
dtype: object

### The Graph is not connected. Let's get the weakly connected components.
`WeaklyConnectedComponents` is one of the available `Graph` algorithms.

The usage pattern always is:
```Python
result = hana_ml.graph.algorithms.<algorithm_name>(graph=<graph_instance>).execute(<parameters>)
```
This returns an instance of the algorithm class, that contains the results as properties (e.g. `result.vertices`). Note: Tabular data are always returned as Pandas `DataFrame`s

In [13]:
import hana_ml.graph.algorithms as hga

wcc = hga.WeaklyConnectedComponents(graph=g_storm).execute()

print(f'There are {wcc.components_count} components in the Graph.')

There are 8332 components in the Graph.


In [14]:
# Which are the largest components (i.e. sub networks)?
wcc.components.sort_values(by='NUMBER_OF_VERTICES', ascending=False).head(2)

,COMPONENT,NUMBER_OF_VERTICES
24,25,3139
4,5,2162


In [15]:
wcc.vertices.head(10)

,ID,COMPONENT
0,WCC_SW000008,1
1,WCC_SW000014,2
2,WCC_SW000021,3
3,WCC_SW000022,4
4,WCC_SW000023,5
5,WCC_SW000032,6
6,WCC_SW000056,7
7,WCC_SW000071,8
8,WCC_SW000085,9
9,WCC_SW000105,10


### Store the component number for each vertex in a HANA Table so we can use it as a filter.
Due to a limitation in GraphQL, `WeaklyConnectedComponents.vertices` can only return the IDs.
If we want to havve the full records, we need to load the information separately.

Options:
- Create Filter List on the client
- Store results to HANA Tale and filter via an select statement

In [16]:
hdf_wcc = create_dataframe_from_pandas(
    connection_context=cc,
    pandas_df=wcc.vertices,
    drop_exist_tab=True,
    table_name='LM_STORMWATER_WCC',
    force=True,
    allow_bigint=True,
    primary_key='ID')

100%|██████████| 2/2 [00:01<00:00,  1.46it/s]


In [17]:
g_storm_comp1 = g_storm.subgraph(
    workspace_name = "LM_STORMWATER_COMP1", 
    vertices_filter='ID IN (SELECT ID FROM LM_STORMWATER_WCC WHERE COMPONENT = 25)',
    force = True
)

In [18]:
g_storm_comp2 = g_storm.subgraph(
    workspace_name = "LM_STORMWATER_COMP2", 
    vertices_filter='ID IN (SELECT ID FROM LM_STORMWATER_WCC WHERE COMPONENT = 5)',
    force = True
)

## Let's plot the two weakly connected components on a map

In [19]:
pdf_storm_comp1_edges = g_storm_comp1.edges_hdf \
    .select('ID', 'SOURCE', 'TARGET', ('SHAPE_GEO.ST_TRANSFORM(4326).ST_ASGEOJSON()', 'GJ')).collect()

pdf_storm_comp2_edges = g_storm_comp2.edges_hdf \
    .select('ID', 'SOURCE', 'TARGET', ('SHAPE_GEO.ST_TRANSFORM(4326).ST_ASGEOJSON()', 'GJ')).collect()

map = KeplerGl(height=600, width=800)
map.add_data(pdf_storm_comp1_edges, 'Stormwater Component 1')
map.add_data(pdf_storm_comp2_edges, 'Stormwater Component 2')
map

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'Stormwater Component 1': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, …

## Let's look Upstream and Downstream

Let's assume somebody reported a problem with the node WCC_SW002719. We want to analyze that further.


In [20]:
start_vertex_id = 'WCC_SW002719'

# Get the details of that vertex
start_vertex = g_storm_comp2.vertices_hdf \
    .filter(f"ID = '{start_vertex_id}'") \
    .select('ID', ('SHAPE_GEO.ST_TRANSFORM(4326).ST_ASGEOJSON()', 'GJ')).collect()

start_vertex

,ID,GJ
0,WCC_SW002719,"{""type"": ""Point"", ""coordinates"": [174.81985879..."


In [21]:
neighbors = hga.Neighbors(graph=g_storm_comp2).execute(
    start_vertex=start_vertex_id, 
    direction='ANY', 
    lower_bound=1, 
    upper_bound=5)

neighbors.vertices.head(5)

,ID
0,WCC_SW009330
1,WCC_SW002721
2,WCC_SW002716
3,WCC_SW002725
4,WCC_SW045665


In [22]:
vkc=g_storm_comp2.vertex_key_column
in_list = neighbors.vertices.ID.str.cat(sep="','")
filter = f"{vkc} IN ('{in_list}')"  # Dynamically build the filter condition as SQL WHERE

print(filter)

pdf_storm_comp2_neighbors = g_storm_comp2.vertices_hdf \
    .filter(filter) \
    .select('ID', ('SHAPE_GEO.ST_TRANSFORM(4326).ST_ASGEOJSON()', 'GJ')).collect()

ID IN ('WCC_SW009330','WCC_SW002721','WCC_SW002716','WCC_SW002725','WCC_SW045665','WCC_SW026945','WCC_SW009327','WCC_SW002697','WCC_SW000612','WCC_SW045666','WCC_SW026956','WCC_SW002720','WCC_SW009295','WCC_SW009328','WCC_SW000611','WCC_SW048148','WCC_SW026955','WCC_SW002723','WCC_SW030461','WCC_SW030458','WCC_SW030459','WCC_SW016959','WCC_SW016960','WCC_SW009329','WCC_SW045667','WCC_SW026966','WCC_SW002722')


In [23]:
map = KeplerGl(height=600, width=800)
map.add_data(pdf_storm_comp2_neighbors, '5-hop neighbors')
map.add_data(start_vertex, 'Start Vertex')
map

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'5-hop neighbors': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18,…

## Upstream and Downstream with NeighborsSubgraphs

In [24]:
g_neighbors_upstream = hga.NeighborsSubgraph(graph=g_storm_comp2).execute(
    start_vertex=start_vertex_id, direction='INCOMING', 
    lower_bound=0, upper_bound=10000)

In [25]:
g_neighbors_downstream = hga.NeighborsSubgraph(graph=g_storm_comp2).execute(
    start_vertex=start_vertex_id, direction='OUTGOING', 
    lower_bound=0, upper_bound=10000)

In [26]:
ekc = g_storm_comp2.edge_key_column

in_list = g_neighbors_upstream.edges.ID.astype(str).str.cat(sep=',' )
pdf_storm_comp2_neighbors_upstream_edges = g_storm_comp2.edges_hdf \
    .filter(f"{ekc} IN ({in_list})") \
    .select('ID', ('SHAPE_GEO.ST_TRANSFORM(4326).ST_ASGEOJSON()', 'GJ')).collect()

in_list = g_neighbors_downstream.edges.ID.astype(str).str.cat(sep=',' )
pdf_storm_comp2_neighbors_downstream_edges = g_storm_comp2.edges_hdf \
    .filter(f"{ekc} IN ({in_list})") \
    .select('ID', ('SHAPE_GEO.ST_TRANSFORM(4326).ST_ASGEOJSON()', 'GJ')).collect()

In [27]:
map = KeplerGl(height=600, width=800)
map.add_data(start_vertex, 'Start Vertex')
map.add_data(pdf_storm_comp2_neighbors_upstream_edges, 'Upstream')
map.add_data(pdf_storm_comp2_neighbors_downstream_edges, 'Downstream')
map

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'Start Vertex': {'index': [0], 'columns': ['ID', 'GJ'], 'data': [['WCC_SW002719', '{"type": "Po…

In [28]:
# The Shortest Path One to All, could give an indication about what to check first
spoa = hga.ShortestPathsOneToAll(graph=g_storm_comp2).execute(source=start_vertex_id, direction='INCOMING', weight='LENGTH_M')

spoa.vertices.sort_values('DISTANCE')

,ID,DISTANCE
12,WCC_SW002719,0.00
14,WCC_SW002721,19.50
42,WCC_SW026945,32.93
52,WCC_SW026956,66.82
13,WCC_SW002720,67.55
...,...,...
76,WCC_SW029634,512.98
78,WCC_SW029636,523.88
79,WCC_SW029637,542.92
80,WCC_SW029638,560.92
